In [20]:
import numpy as np
import pandas as pd
import os
import time
import random
import matplotlib.pyplot as plt 

from glob import glob
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from skimage.feature import hog
from joblib import dump, load
import PIL
import cv2
import pickle
from keras.models import load_model
import time
import os
from difflib import SequenceMatcher
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # Use CPU only
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import keras
import matplotlib.pyplot as plt
import PIL
import openvino as ov

In [7]:
df = pd.read_csv("Validation_data/data.csv")
svc = load('svc_model.joblib') 

In [21]:
core = ov.Core()
model_openvino = core.read_model(model="model.xml")
compiled_model = core.compile_model(model=model_openvino, device_name="CPU")
output_layer = compiled_model.output(0)

In [96]:
def toBool(img):
    pillow = PIL.Image.fromarray(img)
    return np.asarray(pillow.point(lambda x: 0 if x < 128 else 1, mode='1'))
def crop(img_path):
    img =  toBool(np.asarray(PIL.Image.open(img_path)))
    canvas = img.copy()
    ractangle_x_size = 50
    ractangle_y_size = 50
    shape = img.shape
    i_indexes = []
    j_indexes = []
    for i in range(0,shape[1],ractangle_x_size):
        for j in range(0,shape[0],ractangle_y_size):
            ractangle = [[i,j],[i+ractangle_x_size,j+ractangle_y_size]]
            cut_img = img[ractangle[0][1]:ractangle[1][1],ractangle[0][0]:ractangle[1][0]]
            cut_img = cut_img.reshape(cut_img.shape[0]*cut_img.shape[1])
            if(svc.predict([cut_img])[0]>0.5):
                i_indexes.append(i)
                j_indexes.append(j)
    return canvas[min(j_indexes):max(j_indexes)+ractangle_y_size,min(i_indexes):max(i_indexes)+ractangle_x_size]
max_len=25
def decode_batch_of_1(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0][
        :, :max_len
    ]

    output_text = []
    for res in results:

        res = tf.strings.reduce_join(num_to_char(res)).numpy().decode("utf-8")
        output_text.append(res)
    return output_text[0].replace("[UNK]","")
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
characters = ['a', 'ą', 'b', 'c', 'ć', 'd', 'e', 'ę', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'ł',
              'm', 'n', 'ń', 'o', 'ó', 'p', 'r', 's', 'ś', 't', 'u', 'w', 'y', 'z', 'ź', 'ż',' ']
num_to_char = layers.experimental.preprocessing.StringLookup(
    vocabulary=list(characters), mask_token=None, invert=True,num_oov_indices=1
)
img_height = 128
img_width = 496
def encode_single_sample(img_path):
    # 1. Read image
    img = tf.io.read_file(img_path)
    # 2. Decode and convert to grayscale
    img = tf.io.decode_png(img, channels=1)
    # 3. Convert to float32 in [0, 1] range
    img = tf.image.convert_image_dtype(img, tf.float32)
    # 4. Resize to the desired size
    img = tf.image.resize(img, [img_height, img_width])
    # 5. Transpose the image because we want the time
    # dimension to correspond to the width of the image.
    img = tf.transpose(img, perm=[1, 0, 2])
    # 6. Map the characters in label to numbers
    return img


In [104]:
acc = 0
similarity = 0
sample_amount = 1000
for i in range(sample_amount):
    if i%100==0:
        print(i)
    img = crop(df["Img_path"].iloc[i])
    pil = PIL.Image.fromarray(img)
    pil.save("temp/temp.jpg")
    img = encode_single_sample("temp/temp.jpg")
    label_pred = compiled_model([np.array([img])])[output_layer]
    label_pred = decode_batch_of_1(label_pred)
    label = df["Name"].iloc[i] + " "+ df["Surname"].iloc[i]
    label = label.lower()
    if label==label_pred:
        acc+=1
    similarity+= similar(label,label_pred)
print("Accuracy: ",acc/sample_amount)
print("Average similarity: ",similarity/sample_amount)

0
100
200
300
400
500
600
700
800
900
Accuracy:  0.469
Average similarity:  0.9411408277617281
